In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [7]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
# host = 'dev.eye-ai.org'
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-10 17:16:17,142 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 17:16:17,143 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [8]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C60W" # rid dev: 2-BH7T prod: 2-C60W


In [9]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-10 17:16:29,077 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 17:16:29,078 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-10 17:16:29,430 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post18+git.d89701e2.dirty


In [10]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-10 17:16:30,609 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json] transfer successful. 0.60 KB transferred. Elapsed time: 0:00:00.000049.
2024-06-10 17:16:30,610 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json]
2024-06-10 17:16:30,635 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_feature_engineering',
    'rid': '2-C610'}]},
 'execution_rid': '2-C60Y',
 'workflow_rid': '2-C612',
 'bag_paths': [PosixPath('/data/2-7P5P_78d5bc65ac069261e9c19c4fa306c3d1346e7fe9288ad314c94ba36cd41a5b95/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_create_cond_label.json')}

In [11]:
# Compute condition labels
icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])

condition_labels = EA.compute_condition_label(icd10_asso, icd10)

In [14]:
# Insert new condition labels
EA.insert_condition_label(condition_labels)


[{'RID': '2-7XT4', 'Condition_Label': '2-C60M'}, {'RID': '2-7XT6', 'Condition_Label': '2-C60M'}, {'RID': '2-7XT8', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTA', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTC', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTE', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTG', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTJ', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTM', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTP', 'Condition_Label': '2-C60M'}, {'RID': '2-7XTR', 'Condition_Label': '2-C60J'}, {'RID': '2-7XTT', 'Condition_Label': '2-C60J'}, {'RID': '2-7XTW', 'Condition_Label': '2-C60J'}, {'RID': '2-7XTY', 'Condition_Label': '2-C60J'}, {'RID': '2-7XV0', 'Condition_Label': '2-C60J'}, {'RID': '2-7XV2', 'Condition_Label': '2-C60J'}, {'RID': '2-7XV4', 'Condition_Label': '2-C60J'}, {'RID': '2-7XVA', 'Condition_Label': '2-C60J'}, {'RID': '2-7XVE', 'Condition_Label': '2-C60J'}, {'RID': '2-7XVG', 'Condition_Label': '2-C60M'}, {'RID': '2-7XVJ', 'Condition_Label': '2

In [15]:
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

2024-06-10 17:17:15,799 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-06-10 17:17:15,800 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-10 17:17:15,801 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-10 17:17:15,817 - INFO - Checking for updated configuration...
2024-06-10 17:17:15,927 - INFO - Updated configuration found.
2024-06-10 17:17:15,929 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/Execution_Metadata]...
2024-06-10 17:17:15,933 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json].
2024-06-10 17:17:15,933 - INFO - Including file: [/data/lizhiwei/EyeAI_working/Execution_Metadata/Runtime_Env-py